In [1]:
import time
import os
import random
import numpy as np     
import math

import matplotlib.pyplot as plt                                   
import autograd, autograd.core, autograd.extend, autograd.tracer  
import autograd.numpy as anp      
import scipy, scipy.ndimage, scipy.sparse, scipy.sparse.linalg 

from deap import base
from deap import creator
from deap import tools

import random
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

In [2]:
sys.path.append('../../../marine/pipelay_vessel_design_optimisation_using_genetic_algorithms/')

In [3]:
import elitism

In [4]:
from scipy.spatial import ConvexHull

In [5]:
import networkx as nx

In [6]:
x0,y0 = 25, 80

# FEM Model

In [7]:
class ObjectView(object):
    def __init__(self, d): self.__dict__ = d
    
def get_args(normals, forces, density=1e-4):  # Manage the problem setup parameters
    width = normals.shape[0] - 1
    height = normals.shape[1] - 1
    fixdofs = np.flatnonzero(normals.ravel())
    alldofs = np.arange(2 * (width + 1) * (height + 1))
    freedofs = np.sort(list(set(alldofs) - set(fixdofs)))
    params = {
      # material properties
      'young': 1, 'young_min': 1e-9, 'poisson': 0.3, 'g': 0,
      # constraints
      'density': density, 'xmin': 0.001, 'xmax': 1.0,
      # input parameters
      'nelx': width, 'nely': height, 'mask': 1, 'penal': 3.0, 'filter_width': 1,
      'freedofs': freedofs, 'fixdofs': fixdofs, 'forces': forces.ravel(),
      # optimization parameters
      'opt_steps': 80, 'print_every': 10}
    return ObjectView(params)

def mbb_beam(width=y0, height=x0, density=1e-4, y=1, x=0, rd=-1):  # textbook beam example
    normals = np.zeros((width + 1, height + 1, 2))
    normals[0, 0, x] = 1
    normals[0, 0, y] = 1
    normals[0, -1, x] = 1
    normals[0, -1, y] = 1
    forces = np.zeros((width + 1, height + 1, 2))
    forces[-1, rd, y] = -1
    return normals, forces, density

In [8]:
def young_modulus(x, e_0, e_min, p=3):
    return e_min + x ** p * (e_0 - e_min)

def physical_density(x, args, volume_contraint=False, use_filter=True):
    x = args.mask * x.reshape(args.nely, args.nelx)  # reshape from 1D to 2D
    return gaussian_filter(x, args.filter_width) if use_filter else x  # maybe filter

def mean_density(x, args, volume_contraint=False, use_filter=True):
    return anp.mean(physical_density(x, args, volume_contraint, use_filter)) / anp.mean(args.mask)

In [9]:
def objective(x, args, volume_contraint=False, use_filter=True):
    kwargs = dict(penal=args.penal, e_min=args.young_min, e_0=args.young)
    x_phys = physical_density(x, args, volume_contraint=volume_contraint, use_filter=use_filter)
    ke     = get_stiffness_matrix(args.young, args.poisson)  # stiffness matrix
    u      = displace(x_phys, ke, args.forces, args.freedofs, args.fixdofs, **kwargs)
    c      = compliance(x_phys, u, ke, **kwargs)
    return c

In [10]:
@autograd.extend.primitive
def gaussian_filter(x, width): # 2D gaussian blur/filter
    return scipy.ndimage.gaussian_filter(x, width, mode='reflect')

def _gaussian_filter_vjp(ans, x, width): # gives the gradient of orig. function w.r.t. x
    del ans, x  # unused
    return lambda g: gaussian_filter(g, width)
autograd.extend.defvjp(gaussian_filter, _gaussian_filter_vjp)

In [11]:
def compliance(x_phys, u, ke, *, penal=3, e_min=1e-9, e_0=1):
    nely, nelx = x_phys.shape
    ely, elx = anp.meshgrid(range(nely), range(nelx))  # x, y coords for the index map

    n1 = (nely+1)*(elx+0) + (ely+0)  # nodes
    n2 = (nely+1)*(elx+1) + (ely+0)
    n3 = (nely+1)*(elx+1) + (ely+1)
    n4 = (nely+1)*(elx+0) + (ely+1)
    all_ixs = anp.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
    u_selected = u[all_ixs]  # select from u matrix

    ke_u = anp.einsum('ij,jkl->ikl', ke, u_selected)  # compute x^penal * U.T @ ke @ U
    ce = anp.einsum('ijk,ijk->jk', u_selected, ke_u)
    C = young_modulus(x_phys, e_0, e_min, p=penal) * ce.T
    return anp.sum(C)

def get_stiffness_matrix(e, nu):  # e=young's modulus, nu=poisson coefficient
    k = anp.array([1/2-nu/6, 1/8+nu/8, -1/4-nu/12, -1/8+3*nu/8,
                -1/4+nu/12, -1/8-nu/8, nu/6, 1/8-3*nu/8])
    return e/(1-nu**2)*anp.array([[k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
                               [k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
                               [k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
                               [k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
                               [k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
                               [k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
                               [k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
                               [k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]]])

In [12]:
def get_k(stiffness, ke):
    # Constructs sparse stiffness matrix k (used in the displace fn)
    # First, get position of the nodes of each element in the stiffness matrix
    nely, nelx = stiffness.shape
    ely, elx = anp.meshgrid(range(nely), range(nelx))  # x, y coords
    ely, elx = ely.reshape(-1, 1), elx.reshape(-1, 1)

    n1 = (nely+1)*(elx+0) + (ely+0)
    n2 = (nely+1)*(elx+1) + (ely+0)
    n3 = (nely+1)*(elx+1) + (ely+1)
    n4 = (nely+1)*(elx+0) + (ely+1)
    edof = anp.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
    edof = edof.T[0]
    x_list = anp.repeat(edof, 8)  # flat list pointer of each node in an element
    y_list = anp.tile(edof, 8).flatten()  # flat list pointer of each node in elem

    # make the global stiffness matrix K
    kd = stiffness.T.reshape(nelx*nely, 1, 1)
    value_list = (kd * anp.tile(ke, kd.shape)).flatten()
    return value_list, y_list, x_list

def displace(x_phys, ke, forces, freedofs, fixdofs, *, penal=3, e_min=1e-9, e_0=1):
    # Displaces the load x using finite element techniques (solve_coo=most of runtime)
    stiffness = young_modulus(x_phys, e_0, e_min, p=penal)
    k_entries, k_ylist, k_xlist = get_k(stiffness, ke)

    index_map, keep, indices = _get_dof_indices(freedofs, fixdofs, k_ylist, k_xlist)

    u_nonzero = solve_coo(k_entries[keep], indices, forces[freedofs], sym_pos=True)
    u_values = anp.concatenate([u_nonzero, anp.zeros(len(fixdofs))])
    return u_values[index_map]

In [13]:
def _get_dof_indices(freedofs, fixdofs, k_xlist, k_ylist):
    index_map = inverse_permutation(anp.concatenate([freedofs, fixdofs]))
    keep = anp.isin(k_xlist, freedofs) & anp.isin(k_ylist, freedofs)
    # Now we index an indexing array that is being indexed by the indices of k
    i = index_map[k_ylist][keep]
    j = index_map[k_xlist][keep]
    return index_map, keep, anp.stack([i, j])

def inverse_permutation(indices):  # reverses an index operation
    inverse_perm = np.zeros(len(indices), dtype=anp.int64)
    inverse_perm[indices] = np.arange(len(indices), dtype=anp.int64)
    return inverse_perm

In [14]:
def _get_solver(a_entries, a_indices, size, sym_pos):
    # a is (usu.) symmetric positive; could solve 2x faster w/sksparse.cholmod.cholesky(a).solve_A
    a = scipy.sparse.coo_matrix((a_entries, a_indices), shape=(size,)*2).tocsc()
    return scipy.sparse.linalg.splu(a).solve

@autograd.primitive
def solve_coo(a_entries, a_indices, b, sym_pos=False):
    solver = _get_solver(a_entries, a_indices, b.size, sym_pos)
    return solver(b)

def grad_solve_coo_entries(ans, a_entries, a_indices, b, sym_pos=False):
    def jvp(grad_ans):
        lambda_ = solve_coo(a_entries, a_indices if sym_pos else a_indices[::-1],
                            grad_ans, sym_pos)
        i, j = a_indices
        return -lambda_[i] * ans[j]
    return jvp

autograd.extend.defvjp(solve_coo, grad_solve_coo_entries,
                       lambda: print('err: gradient undefined'),
                       lambda: print('err: gradient not implemented'))

In [15]:
def fast_stopt(args, x):

    reshape = lambda x: x.reshape(args.nely, args.nelx)
    objective_fn = lambda x: objective(reshape(x), args)
#     constraint = lambda params: mean_density(reshape(params), args) - args.density
    constraint = lambda params: mean_density(reshape(params), args) 
    value = objective_fn(x)
    const = constraint(x)
    return value, const

In [16]:
def draw(X):  
    plt.figure(dpi=50) 
    print('\nFinal Cantilever beam design:')
    plt.imshow(X) 
    plt.show()

# Genetic Modelling

In [17]:
# Genetic Algorithm constants:
POPULATION_SIZE = 300
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.1   # probability for mutating an individual
MAX_GENERATIONS = 10000
HALL_OF_FAME_SIZE = 30

In [18]:
PENALTY_VALUE = 1000.0    

In [19]:
# set the random seed:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [20]:
toolbox = base.Toolbox()

In [21]:
# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

In [22]:
# create the Individual class based on list:
creator.create("Individual", list, fitness=creator.FitnessMin)

In [23]:
# create an operator that randomly returns 0 or 1:
toolbox.register("zeroOrOne", random.randint, 0, 1)

In [24]:
# create the individual operator to fill up an Individual instance:
toolbox.register("individualCreator", 
                 tools.initRepeat,
                 creator.Individual,
                 toolbox.zeroOrOne, 
                 x0*y0+4)

In [25]:
# create the population operator to generate a list of individuals:
toolbox.register("populationCreator", 
                 tools.initRepeat, 
                 list, 
                 toolbox.individualCreator)

In [26]:
# convert a binary list to decimal
def binatodeci(binary):
    return sum(val*(2**idx) for idx, val in enumerate(reversed(binary)))

In [27]:
# fitness calculation
def staticFEM(individual):
    
    args = get_args(*mbb_beam(rd=-1))
    
    x = np.array([t if t==1 else 1e-4 for t in individual[:-4]]) 
    
    try:
        tmp, const = fast_stopt(args, x)
        if const>(55+binatodeci(individual[-4:]))/100:
            return PENALTY_VALUE, 
    except:
        return PENALTY_VALUE, 
    
    
    return tmp,  # return a tuple

In [28]:
toolbox.register("evaluate", staticFEM)

# genetic operators:
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/(x0*y0+4))

In [29]:
# Genetic Algorithm flow:
def main():

    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # prepare the statistics object:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

    # perform the Genetic Algorithm flow with hof feature added:
    population, logbook = elitism.eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                              ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=True)

    # print best solution found:
    best = hof.items[0]
    print("-- Best Individual = ", best)
    print("-- Best Fitness = ", best.fitness.values[0])
    print()
    
    x = np.array(best[:-4]).reshape(x0,y0)
    
    draw(x)
    

    # extract statistics:
    minFitnessValues, meanFitnessValues = logbook.select("min", "avg")

    # plot statistics:
    sns.set_style("whitegrid")
    plt.plot(minFitnessValues, color='red')
    plt.plot(meanFitnessValues, color='green')
  
    plt.xlabel('Generation')
    plt.ylabel('Min / Average Fitness')
    plt.title('Min and Average fitness over Generations')
    plt.show()    

In [ ]:
main()

gen	nevals	min    	avg        
0  	300   	1519.41	1.30202e+06
1  	248   	1497.31	4709.67    
2  	252   	1391.11	2367.04    
3  	242   	1342.01	2836.64    
4  	248   	1321.37	1656.15    
5  	249   	1250.22	1536.82    
6  	244   	1244.01	1469.6     
7  	249   	1179.31	1407.72    
8  	254   	1135.04	1354.05    
9  	244   	1133.18	1292.18    
10 	257   	1072.74	1234.25    
11 	252   	1033.18	1194.59    
12 	237   	996.745	1153.91    
13 	239   	971.986	1119.52    
14 	259   	929.71 	1083.33    
15 	261   	901.267	1047.47    
16 	238   	871.115	1015.68    
17 	248   	871.115	981.758    
18 	241   	842.095	951.205    
19 	250   	806.54 	923.877    
20 	240   	803.652	898.855    
21 	243   	794.54 	877.302    
22 	247   	787.556	849.838    
23 	248   	773.765	830.019    
24 	237   	762.001	812.518    
25 	253   	762.001	800.955    
26 	240   	748.985	791.746    
27 	251   	730.211	779.378    
28 	237   	728.014	769.965    
29 	246   	723.226	760.269    
30 	252   	711.096	749.492    
31 	250 

264	253   	444.182	504.123    
265	253   	444.182	516.722    
266	248   	444.144	523.898    
267	244   	443.72 	527.252    
268	232   	443.072	517.669    
269	246   	443.072	528.536    
270	250   	442.484	543.212    
271	257   	442.295	533.762    
272	241   	441.77 	538.955    
273	237   	441.522	525.637    
274	232   	441.383	519.821    
275	246   	440.563	528.751    
276	250   	440.563	504.733    
277	245   	440.4  	502.094    
278	240   	440.4  	527.781    
279	233   	439.869	508.804    
280	253   	439.72 	521.608    
281	261   	439.268	521.301    
282	240   	437.193	496.811    
283	259   	437.193	507.646    
284	245   	436.49 	501.7      
285	254   	435.747	488.375    
286	243   	435.193	504.653    
287	253   	435.042	502.405    
288	241   	434.876	498.072    
289	241   	434.399	487.869    
290	247   	434.399	489.189    
291	243   	434.399	494.335    
292	243   	434.061	501.495    
293	239   	433.771	495.537    
294	240   	433.584	493.387    
295	243   	432.806	491.276    
296	244 

529	251   	327.298	328.95     
530	245   	327.298	328.623    
531	241   	326.7  	328.307    
532	248   	326.7  	327.961    
533	255   	325.564	327.666    
534	238   	324.914	327.343    
535	249   	324.585	326.949    
536	237   	324.585	326.539    
537	249   	323.567	326.096    
538	240   	323.225	325.603    
539	245   	323.225	325.187    
540	243   	323.225	324.698    
541	243   	323.225	324.366    
542	252   	322.831	324.067    
543	245   	322.558	323.771    
544	243   	322.243	323.55     
545	251   	321.864	323.218    
546	258   	321.679	322.971    
547	242   	321.329	322.689    
548	237   	321.047	322.449    
549	257   	320.888	322.24     
550	241   	320.817	321.969    
551	232   	320.379	321.712    
552	243   	320.248	321.504    
553	248   	320.139	321.149    
554	244   	319.327	323.131    
555	241   	319.314	320.529    
556	250   	318.944	320.194    
557	255   	318.749	319.923    
558	251   	318.179	319.595    
559	255   	317.958	319.604    
560	250   	317.666	318.984    
561	246 

794	252   	295.923	378.327    
795	251   	295.871	382.982    
796	244   	295.823	380.592    
797	246   	295.823	373.587    
798	255   	295.765	378.194    
799	249   	295.378	385.213    
800	241   	295.378	378.124    
801	246   	295.378	385.142    
802	241   	295.378	359.319    
803	255   	295.316	413.27     
804	244   	295.3  	396.797    
805	254   	295.142	375.609    
806	248   	295.078	410.774    
807	253   	294.919	396.626    
808	248   	294.91 	368.377    
809	251   	294.84 	384.704    
810	228   	294.818	386.962    
811	239   	294.814	400.973    
812	248   	294.78 	393.859    
813	240   	294.78 	407.887    
814	249   	294.78 	379.662    
815	245   	294.639	372.578    
816	242   	294.481	374.899    
817	242   	294.481	377.21     
818	254   	294.481	363.087    
819	254   	294.449	414.766    
820	250   	294.349	400.778    
821	250   	294.238	381.835    
822	248   	294.105	391.132    
823	243   	293.908	400.482    
824	249   	293.908	400.493    
825	235   	293.908	381.567    
826	245 

1057	251   	286.406	403.312    
1058	232   	286.406	422.262    
1059	242   	286.406	393.741    
1060	246   	286.406	400.909    
1061	251   	286.406	388.976    
1062	238   	286.385	405.572    
1063	252   	286.319	400.77     
1064	241   	286.279	379.344    
1065	244   	286.279	393.661    
1066	238   	286.279	398.359    
1067	251   	286.279	395.948    
1068	248   	286.212	381.707    
1069	247   	286.212	424.5      
1070	250   	286.212	398.293    
1071	262   	286.212	417.293    
1072	239   	286.139	403.048    
1073	246   	286.139	393.504    
1074	255   	286.128	410.102    
1075	244   	286.061	412.448    
1076	245   	286.036	402.93     
1077	243   	286.036	393.38     
1078	239   	285.946	390.966    
1079	252   	285.946	405.263    
1080	251   	285.914	407.588    
1081	242   	285.914	386.144    
1082	244   	285.914	383.728    
1083	246   	285.895	402.746    
1084	249   	285.895	400.383    
1085	244   	285.895	393.205    
1086	242   	285.86 	383.672    
1087	253   	285.844	367.003    
1088	248

...

6968	250   	260.222	275.047    
6969	254   	260.222	277.563    
6970	240   	260.222	272.569    
6971	245   	260.222	267.656    
6972	251   	260.222	265.193    
6973	252   	260.222	272.565    
6974	231   	260.222	267.652    
6975	256   	260.222	275.063    
6976	252   	260.222	267.651    
6977	250   	260.222	267.643    
6978	226   	260.222	272.569    
6979	248   	260.222	277.559    
6980	254   	260.222	279.991    
6981	252   	260.222	272.617    
6982	240   	260.222	275.036    
6983	247   	260.222	272.577    
6984	236   	260.222	270.419    
6985	239   	260.222	275.206    
6986	234   	260.222	272.572    
6987	237   	260.222	262.717    
6988	245   	260.222	275.048    
6989	230   	260.222	275.059    
6990	247   	260.222	279.967    
6991	247   	260.222	267.658    
6992	249   	260.222	275.084    
6993	241   	260.222	270.121    
6994	233   	260.222	267.668    
6995	246   	260.222	275.037    
6996	233   	260.222	267.642    
6997	247   	260.222	275.033    
6998	246   	260.222	275.029    
6999	255

7225	251   	260.124	348.958    
7226	248   	260.124	334.593    
7227	256   	260.124	353.896    
7228	247   	260.124	351.425    
7229	248   	260.124	351.486    
7230	255   	260.124	336.636    
7231	250   	260.124	324.328    
7232	244   	260.124	334.152    
7233	244   	260.124	299.763    
7234	241   	260.124	299.634    
7235	235   	260.12 	292.227    
7236	238   	260.12 	279.895    
7237	248   	260.12 	272.479    
7238	250   	260.12 	270.328    
7239	241   	260.12 	277.418    
7240	247   	260.12 	277.416    
7241	249   	260.12 	284.817    
7242	249   	260.12 	277.42     
7243	243   	260.12 	274.939    
7244	246   	260.12 	284.899    
7245	239   	260.12 	279.886    
7246	254   	260.12 	287.293    
7247	238   	260.12 	289.755    
7248	249   	260.12 	289.78     
7249	238   	260.12 	284.832    
7250	233   	260.12 	292.264    
7251	245   	260.12 	297.15     
7252	254   	260.12 	279.896    
7253	247   	260.12 	287.279    
7254	255   	260.12 	284.826    
7255	252   	260.12 	285.133    
7256	237

7482	253   	260.054	279.81     
7483	252   	260.054	265.07     
7484	235   	260.054	277.338    
7485	244   	260.054	289.668    
7486	258   	260.054	272.401    
7487	241   	260.054	265.028    
7488	243   	260.054	265.007    
7489	247   	260.054	279.804    
7490	236   	260.054	274.866    
7491	242   	260.054	272.402    
7492	241   	260.054	265.001    
7493	246   	260.054	272.401    
7494	253   	260.054	269.945    
7495	239   	260.054	277.341    
7496	245   	260.054	272.407    
7497	246   	260.054	269.939    
7498	240   	260.054	269.938    
7499	239   	260.054	274.88     
7500	247   	260.054	279.823    
7501	255   	260.054	297.085    
7502	248   	260.054	267.478    
7503	241   	260.054	279.813    
7504	246   	260.054	282.277    
7505	235   	260.054	289.681    
7506	241   	260.054	279.82     
7507	241   	260.054	284.737    
7508	248   	260.054	294.632    
7509	242   	260.054	282.304    
7510	247   	260.054	277.351    
7511	243   	260.054	282.331    
7512	243   	260.054	279.836    
7513	230

7739	251   	260.047	282.29     
7740	247   	260.047	277.349    
7741	241   	260.047	292.132    
7742	246   	260.047	306.957    
7743	249   	260.047	282.346    
7744	243   	260.047	287.205    
7745	248   	260.047	287.442    
7746	251   	260.047	279.936    
7747	247   	260.047	302.009    
7748	247   	260.047	316.844    
7749	246   	260.047	289.683    
7750	249   	260.047	299.538    
7751	246   	260.047	277.335    
7752	243   	260.047	299.549    
7753	253   	260.047	299.537    
7754	247   	260.047	302.009    
7755	250   	260.047	326.689    
7756	234   	260.047	289.684    
7757	244   	260.047	309.424    
7758	247   	260.047	314.337    
7759	233   	260.047	334.078    
7760	245   	260.047	319.34     
7761	249   	260.047	336.602    
7762	239   	260.047	336.638    
7763	232   	260.047	339.029    
7764	240   	260.047	344.046    
7765	240   	260.047	322.136    
7766	242   	260.047	324.218    
7767	238   	260.047	309.423    
7768	248   	260.04 	324.534    
7769	249   	260.04 	297.073    
7770	251

7996	244   	259.997	279.762    
7997	243   	259.997	287.163    
7998	240   	259.997	284.719    
7999	242   	259.997	277.311    
8000	249   	259.996	284.742    
8001	244   	259.996	284.709    
8002	237   	259.996	277.315    
8003	243   	259.996	287.279    
8004	253   	259.996	279.753    
8005	248   	259.996	279.764    
8006	240   	259.996	279.756    
8007	243   	259.996	292.086    
8008	255   	259.996	269.895    
8009	238   	259.996	279.752    
8010	247   	259.996	277.284    
8011	249   	259.996	277.294    
8012	246   	259.996	272.358    
8013	251   	259.996	269.931    
8014	240   	259.996	269.905    
8015	241   	259.996	272.351    
8016	258   	259.996	272.375    
8017	249   	259.996	274.831    
8018	247   	259.996	279.753    
8019	248   	259.996	267.419    
8020	243   	259.996	269.908    
8021	250   	259.996	272.367    
8022	250   	259.996	274.907    
8023	240   	259.996	282.228    
8024	250   	259.996	274.834    
8025	244   	259.996	279.772    
8026	235   	259.996	279.774    
8027	243